In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

print(keras.__version__)

2.9.0
2.9.0


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [6]:
#this data have been already cleaned and process please read "cleaned_data.csv" instead of "age_gender.csv"
data_set = pd.read_csv("age_gender.csv")

In [7]:
split_pixels = data_set["pixels"].str.split(' ', expand = True)
data_set.drop("img_name",axis=1,inplace=True)

In [8]:
result = pd.concat([data_set, split_pixels], axis = 1)

In [14]:
split_pixels = split_pixels.apply(pd.to_numeric, args = ('ignore', 'float'))

In [18]:
pom = split_pixels / 255
pom.head()

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,0.505882,0.501961,0.501961,0.494118,0.498039,0.509804,0.521569,0.529412,0.545098,0.556863,...,0.482353,0.505882,0.525490,0.541176,0.556863,0.564706,0.564706,0.572549,0.572549,0.572549
1,0.643137,0.290196,0.435294,0.658824,0.662745,0.670588,0.686275,0.713726,0.721569,0.737255,...,0.658824,0.741176,0.874510,0.945098,0.925490,0.909804,0.800000,0.713726,0.666667,0.580392
2,0.262745,0.274510,0.278431,0.274510,0.270588,0.262745,0.274510,0.309804,0.352941,0.403922,...,0.458824,0.443137,0.454902,0.454902,0.458824,0.458824,0.450980,0.439216,0.435294,0.423529
3,0.756863,0.772549,0.776471,0.784314,0.780392,0.784314,0.792157,0.796078,0.800000,0.803922,...,0.854902,0.870588,0.890196,0.894118,0.894118,0.901961,0.905882,0.921569,0.937255,0.964706
4,0.792157,0.803922,0.819608,0.823529,0.819608,0.819608,0.823529,0.827451,0.831373,0.839216,...,0.647059,0.654902,0.654902,0.654902,0.658824,0.670588,0.670588,0.654902,0.639216,0.631373


In [19]:
result = pd.concat([data_set, pom], axis = 1)

In [20]:
result.to_csv("cleaned_data.csv")
#end of cleaning portion

In [21]:
#start here
data_set = pd.read_csv("cleaned_data.csv")
data_set.drop("age",axis=1,inplace=True) ; data_set.drop("ethnicity",axis=1,inplace=True) ; data_set.drop("pixels",axis=1,inplace=True)
data_set.head()

,gender,0,1,2,3,4,5,6,7,8,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,0,0.505882,0.501961,0.501961,0.494118,0.498039,0.509804,0.521569,0.529412,0.545098,...,0.482353,0.505882,0.525490,0.541176,0.556863,0.564706,0.564706,0.572549,0.572549,0.572549
1,0,0.643137,0.290196,0.435294,0.658824,0.662745,0.670588,0.686275,0.713726,0.721569,...,0.658824,0.741176,0.874510,0.945098,0.925490,0.909804,0.800000,0.713726,0.666667,0.580392
2,0,0.262745,0.274510,0.278431,0.274510,0.270588,0.262745,0.274510,0.309804,0.352941,...,0.458824,0.443137,0.454902,0.454902,0.458824,0.458824,0.450980,0.439216,0.435294,0.423529
3,0,0.756863,0.772549,0.776471,0.784314,0.780392,0.784314,0.792157,0.796078,0.800000,...,0.854902,0.870588,0.890196,0.894118,0.894118,0.901961,0.905882,0.921569,0.937255,0.964706
4,0,0.792157,0.803922,0.819608,0.823529,0.819608,0.819608,0.823529,0.827451,0.831373,...,0.647059,0.654902,0.654902,0.654902,0.658824,0.670588,0.670588,0.654902,0.639216,0.631373


In [24]:
data_set.shape

(23705, 2305)

In [25]:
Model_working_set = data_set.sample(frac = 0.3)
train = Model_working_set.sample(frac = 0.8)
test = Model_working_set.drop(train.index) 

In [26]:
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'.\\my_logs\\run_2022_05_31-18_57_32'

In [27]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.callbacks import History

In [37]:
X_train = train.drop("gender", axis = 1)
y_train = train.gender
X_test = test.drop("gender", axis = 1)
y_test = test.gender

In [45]:
X_train.shape[1]

2304

In [43]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [44]:
model = tf.keras.models.Sequential()

In [48]:
model.add(Dense(2500, activation = "sigmoid", input_shape =(X_train.shape[1], )))
model.add(Dense(1000, activation = "elu" ))
model.add(Dense(500, activation = "elu" ))
model.add(Dense(100, activation = "elu" ))
model.add(Dense(10, activation = "elu" ))
model.add(Dense(1, activation = "sigmoid" ))

In [49]:
model.compile(loss="binary_crossentropy",optimizer="Adam", metrics=["accuracy"])

In [50]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

Epoch 1/100
178/178 [==============================] - 384s 119ms/step - loss: 0.7384 - accuracy: 0.5870 - val_loss: 0.5672 - val_accuracy: 0.7328
Epoch 2/100
178/178 [==============================] - 18s 101ms/step - loss: 0.5561 - accuracy: 0.7237 - val_loss: 0.4966 - val_accuracy: 0.7932
Epoch 3/100
178/178 [==============================] - 18s 101ms/step - loss: 0.5035 - accuracy: 0.7685 - val_loss: 0.4635 - val_accuracy: 0.7911
Epoch 4/100
178/178 [==============================] - 18s 102ms/step - loss: 0.4457 - accuracy: 0.7949 - val_loss: 0.4331 - val_accuracy: 0.8186
Epoch 5/100
178/178 [==============================] - 19s 105ms/step - loss: 0.4361 - accuracy: 0.7963 - val_loss: 0.4217 - val_accuracy: 0.8087
Epoch 6/100
178/178 [==============================] - 17s 96ms/step - loss: 0.3900 - accuracy: 0.8236 - val_loss: 0.4097 - val_accuracy: 0.8150
Epoch 7/100
178/178 [==============================] - 17s 97ms/step - loss: 0.3881 - accuracy: 0.8218 - val_loss: 0.5499 - 

178/178 [==============================] - 18s 104ms/step - loss: 0.2684 - accuracy: 0.8770 - val_loss: 0.4027 - val_accuracy: 0.8284
Epoch 58/100
178/178 [==============================] - 18s 101ms/step - loss: 0.2534 - accuracy: 0.8835 - val_loss: 0.3786 - val_accuracy: 0.8495
Epoch 59/100
178/178 [==============================] - 17s 96ms/step - loss: 0.2571 - accuracy: 0.8842 - val_loss: 0.4366 - val_accuracy: 0.7897
Epoch 60/100
178/178 [==============================] - 17s 95ms/step - loss: 0.2575 - accuracy: 0.8800 - val_loss: 0.3601 - val_accuracy: 0.8460
Epoch 61/100
178/178 [==============================] - 17s 96ms/step - loss: 0.2320 - accuracy: 0.8965 - val_loss: 0.4062 - val_accuracy: 0.8453
Epoch 62/100
178/178 [==============================] - 18s 100ms/step - loss: 0.2575 - accuracy: 0.8819 - val_loss: 0.3725 - val_accuracy: 0.8368
Epoch 63/100
178/178 [==============================] - 18s 103ms/step - loss: 0.2411 - accuracy: 0.8900 - val_loss: 0.3556 - val_accu

In [51]:
model.save("./trained_models/1")

INFO:tensorflow:Assets written to: ./trained_models/1\assets
